In [1]:
from tensorflow.keras.models import load_model
from pickle import load
import pandas as pd
import numpy as np

In [6]:
model = load_model('model.keras')
with open('geo_encoder.pkl', 'rb') as fp: label_encoder_geo = load(fp)
with open('scaler.pkl', 'rb') as fp: scaler = load(fp)

C:\Users\ASUS\anaconda3\envs\customer-churn\lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [61]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [35]:
country_encoded = label_encoder_geo.transform(pd.DataFrame({'Geography': [input_data['Geography']]}))

In [36]:
geo_encoded_df = pd.DataFrame(country_encoded, columns=label_encoder_geo.get_feature_names_out())

In [37]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [62]:
input_data = pd.DataFrame([input_data])

In [63]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [64]:
input_data = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)

In [65]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [66]:
input_data.Gender = input_data.Gender.map({'Male': 1, 'Female': 0})

In [67]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [68]:
input_data.drop('Geography', axis=1, inplace=True)

In [69]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [75]:
input_df = scaler.transform(input_data)

In [76]:
input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [78]:
prediction = model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


In [79]:
prediction

array([[0.02592683]], dtype=float32)

In [82]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.025926834)

In [83]:
if prediction_proba > 0.5: print("Customer will churn.")
else: print("Customer will not churn.")

Customer will not churn.
